Read the sales data Excel sheet. There are 4 rows preceeding the header

In [7]:
import pandas as pd

sales = pd.read_excel('../data/1_original/rollingsales_brooklyn.xls', header=4)
sales

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,3,BATH BEACH,01 ONE FAMILY DWELLINGS,1,6359,70,NaN,S1,8684 15TH AVENUE,NaN,...,1.0,1.0,2.0,1933.0,4080.0,1930.0,1,S1,1300000,2020-04-28
1,3,BATH BEACH,01 ONE FAMILY DWELLINGS,1,6360,48,NaN,A5,14 BAY 10TH STREET,NaN,...,1.0,0.0,1.0,2513.0,1428.0,1930.0,1,A5,849000,2020-03-18
2,3,BATH BEACH,01 ONE FAMILY DWELLINGS,1,6367,24,NaN,A9,8645 BAY 16TH STREET,NaN,...,1.0,0.0,1.0,1571.0,1456.0,1935.0,1,A9,830000,2020-06-26
3,3,BATH BEACH,01 ONE FAMILY DWELLINGS,1,6371,19,NaN,A9,79 BAY 20TH STREET,NaN,...,1.0,0.0,1.0,2320.0,1566.0,1930.0,1,A9,0,2020-02-21
4,3,BATH BEACH,01 ONE FAMILY DWELLINGS,1,6371,60,NaN,A9,8620 19TH AVENUE,NaN,...,1.0,0.0,1.0,2417.0,2106.0,1930.0,1,A9,1188000,2020-07-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18470,3,WYCKOFF HEIGHTS,41 TAX CLASS 4 - OTHER,4,3407,21,NaN,Z9,376 WEIRFIELD STREET,NaN,...,0.0,0.0,0.0,10058.0,0.0,NaN,4,Z9,3965000,2020-10-30
18471,3,WYCKOFF HEIGHTS,41 TAX CLASS 4 - OTHER,4,3407,21,NaN,Z9,376 WEIRFIELD STREET,NaN,...,0.0,0.0,0.0,10058.0,0.0,NaN,4,Z9,0,2020-08-19
18472,3,WYCKOFF HEIGHTS,41 TAX CLASS 4 - OTHER,4,3407,26,NaN,Z9,378 WEIRFIELD STREET,NaN,...,0.0,0.0,0.0,2150.0,0.0,NaN,4,Z9,3965000,2020-10-30
18473,3,WYCKOFF HEIGHTS,44 CONDO PARKING,4,3328,1086,NaN,RP,"358 GROVE ST, P25",P25,...,NaN,1.0,1.0,NaN,NaN,NaN,4,RP,1,2020-03-09


Filter out 
- garages, offices, etc (keep only familiy dwelling) 
- entries with price of $10 and below (assuming if one gets a dwelling gifted, this buyer guide is not used)
- entries with gross square feet of zero

In [8]:
building_classes = ["01 ONE FAMILY DWELLINGS", "02 TWO FAMILY DWELLINGS", "03 THREE FAMILY DWELLINGS"]

sales = sales[(sales["BUILDING CLASS CATEGORY"].isin(building_classes)) & (sales["SALE PRICE"] > 10) & (sales["GROSS SQUARE FEET"] > 0)].copy()

Get rid of unused columns

In [9]:
sales.drop(columns=["BOROUGH","NEIGHBORHOOD","TAX CLASS AT PRESENT","BLOCK","LOT","EASE-MENT","BUILDING CLASS AT PRESENT","APARTMENT NUMBER","TOTAL UNITS","TAX CLASS AT TIME OF SALE","BUILDING CLASS AT TIME OF SALE","SALE DATE", "RESIDENTIAL UNITS","COMMERCIAL UNITS","ADDRESS","LAND SQUARE FEET", "YEAR BUILT", "BUILDING CLASS CATEGORY"], inplace=True)

sales

,ZIP CODE,GROSS SQUARE FEET,SALE PRICE
0,11228,4080.0,1300000
1,11228,1428.0,849000
2,11214,1456.0,830000
4,11214,2106.0,1188000
5,11228,1804.0,975000
...,...,...,...
18407,11237,2940.0,1075000
18414,11237,2700.0,1035000
18415,11237,2700.0,1300000
18416,11237,2700.0,980000


Calculate a new column: Price per gross square feet
(see https://home.akitabox.com/blog/gross-square-footage-definitions)

In [10]:
sales["PRICE PER GSQ"] = sales["SALE PRICE"] / sales["GROSS SQUARE FEET"]
sales

,ZIP CODE,GROSS SQUARE FEET,SALE PRICE,PRICE PER GSQ
0,11228,4080.0,1300000,318.627451
1,11228,1428.0,849000,594.537815
2,11214,1456.0,830000,570.054945
4,11214,2106.0,1188000,564.102564
5,11228,1804.0,975000,540.465632
...,...,...,...,...
18407,11237,2940.0,1075000,365.646259
18414,11237,2700.0,1035000,383.333333
18415,11237,2700.0,1300000,481.481481
18416,11237,2700.0,980000,362.962963


group the sales by ZIP CODE. And then index must be removed, otherwise Folium/Colorpleth gets into troubles later

In [11]:
sales_grouped = sales.groupby(['ZIP CODE']).mean()
sales_grouped = sales_grouped.reset_index()
sales_grouped.to_csv('./../data/3_prepared/sales_grouped.csv', ';')
sales_grouped.head()

,ZIP CODE,GROSS SQUARE FEET,SALE PRICE,PRICE PER GSQ
0,11201,3390.250000,7.172004e+06,2223.469918
1,11203,1753.069767,6.171186e+05,367.874395
2,11204,2043.782313,1.155987e+06,608.351331
3,11205,2842.794872,1.894212e+06,703.602411
4,11206,2760.326087,1.845439e+06,677.738777


Now, filter out all ZIP entries which are not in the sales (-> not in Brooklyn). Otherwise Folium/Colorpleth gets into troubles later 

In [12]:
import geopandas as gpd

all_zip_codes = gpd.read_file('./../data/2_intermediate/ZIP_codes.geojson')
all_zip_codes['ZIPCODE'] = pd.to_numeric(all_zip_codes['ZIPCODE'])
brooklyn_zip_codes = all_zip_codes[(all_zip_codes["ZIPCODE"].isin(sales_grouped['ZIP CODE']))].copy()
brooklyn_zip_codes.to_file('./../data/3_prepared/ZIP_codes_BrooklynOnly.geojson', driver='GeoJSON')
brooklyn_zip_codes.head(3)

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.93740 40.67973, -73.93487 40.679..."
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.90294 40.67084, -73.90223 40.668..."
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.95797 40.67066, -73.95576 40.670..."
